# MODEL


In [ ]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# import data from github
url = 'https://raw.githubusercontent.com/SankarSivan/Stock-Price-Prediction-Apple/main/AAPL.csv'
df = pd.read_csv(url)

In [ ]:
# Step 2: Load & Clean Data
url = 'https://raw.githubusercontent.com/SankarSivan/Stock-Price-Prediction-Apple/main/AAPL.csv'
df = pd.read_csv(url, parse_dates=["Date"], index_col="Date")
df = df[['Adj Close']].rename(columns={"Adj Close": "adj_close"})
df.fillna(method="ffill", inplace=True)
df.head()

In [ ]:
# Step 3: Create Sequences for Time Series Forecasting
SEQ_LEN = 60
def create_sequences(data, gap=1):
    X, y = [], []
    for i in range(len(data) - SEQ_LEN - gap + 1):
        X.append(data[i:i+SEQ_LEN])
        y.append(data[i+SEQ_LEN+gap-1])
    return np.array(X), np.array(y)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

X, y = create_sequences(scaled, gap=1)
X = X.reshape((X.shape[0], X.shape[1], 1))  # (samples, timesteps, features)

split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

In [ ]:
# Step 4: Train SimpleRNN
def build_rnn():
    model = Sequential([
        SimpleRNN(50, input_shape=(SEQ_LEN, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

rnn_model = build_rnn()
es = EarlyStopping(patience=5, restore_best_weights=True)
rnn_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[es])


In [ ]:
rnn_model.summary()

In [ ]:
# Step 5: Train LSTM (basic version)
def build_lstm(units=50, lr=0.001):
    model = Sequential([
        LSTM(units, input_shape=(SEQ_LEN, 1)),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse')
    return model

lstm_model = build_lstm()
lstm_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[es])


In [ ]:
lstm_model.summary()

In [ ]:
# Step 6: Manual Hyperparameter Tuning for LSTM
param_grid = {
    "units": [32, 64],
    "lr": [0.001, 0.005],
    "epochs": [10, 20]
}
results = []
best_rmse = float("inf")
best_model = None

for units, lr, ep in product(*param_grid.values()):
    print(f"Training: units={units}, lr={lr}, epochs={ep}")
    model = build_lstm(units, lr)
    model.fit(X_train, y_train, epochs=ep, batch_size=32, validation_data=(X_val, y_val), callbacks=[es], verbose=0)
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    results.append((units, lr, ep, rmse))

    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_params = (units, lr, ep)

print("Best:", best_params, "RMSE:", best_rmse)
best_model.save("lstm_manual.h5")


In [ ]:
# Step 7: Evaluate and Plot
def inverse_scale(y_scaled):
    y_pad = np.hstack([y_scaled, np.zeros((len(y_scaled), 1))])
    return scaler.inverse_transform(y_pad)[:, 0]

val_pred = best_model.predict(X_val)
val_true_inv = inverse_scale(y_val.reshape(-1, 1))
val_pred_inv = inverse_scale(val_pred)

plt.figure(figsize=(12,5))
plt.plot(val_true_inv, label="Actual Price")
plt.plot(val_pred_inv, label="Predicted Price")
plt.title("Prediction vs Actual")
plt.legend()
plt.show()

mae = mean_absolute_error(val_true_inv, val_pred_inv)
rmse = np.sqrt(mean_squared_error(val_true_inv, val_pred_inv))
print(f"MAE: {mae:.2f} | RMSE: {rmse:.2f}")


In [ ]:
# Step 7: Evaluate and Plot
def inverse_scale(y_scaled):
    y_pad = np.hstack([y_scaled, np.zeros((len(y_scaled), 1))])
    return scaler.inverse_transform(y_pad)[:, 0]

val_pred = best_model.predict(X_val)
val_true_inv = inverse_scale(y_val.reshape(-1, 1))
val_pred_inv = inverse_scale(val_pred)

plt.figure(figsize=(12,5))
plt.plot(val_true_inv, label="Actual Price")
plt.plot(val_pred_inv, label="Predicted Price")
plt.title("Prediction vs Actual")
plt.legend()
plt.show()

mae = mean_absolute_error(val_true_inv, val_pred_inv)
rmse = np.sqrt(mean_squared_error(val_true_inv, val_pred_inv))
print(f"MAE: {mae:.2f} | RMSE: {rmse:.2f}")

In [ ]:
best_model.summary()